# Projekt Analiza Danych
## AKADEMIA LEONA KOŹMIŃSKIEGO
## Temat: **Analiza danych dotyczących wypadków samochodowych w Nowym Jorku**

---

# 📘 **Wstęp do projektu – Część I**

> Celem niniejszej części projektu jest przygotowanie i wstępne oczyszczenie danych dotyczących wypadków drogowych w Nowym Jorku. Dane te stanowią podstawę do przeprowadzenia dalszych analiz przestrzennych, czasowych i statystycznych.


### 🔍 **Założenia analizy**

📌 **Zakres danych**:  
Analizowany zbiór danych dotyczy zdarzeń drogowych na terenie Nowego Jorku, rejestrowanych przez lokalne służby.

📌 **Granice analizy przestrzennej**:  
Jeżeli dane lokalizacyjne wskazują na położenie **poza granicami miasta**, należy uznać to za błąd w danych. Takie przypadki:
- ✅ **uwzględniamy** w ogólnych analizach (np. liczba wypadków ogółem),
- ❌ **pomijamy** w analizach przestrzennych i dzielnicowych, gdzie lokalizacja ma kluczowe znaczenie.

📌 **Cel cz. I**:  
- Ocena jakości danych wejściowych,  
- Eliminacja braków i błędów,  
- Wstępne przygotowanie danych do dalszych analiz.


---
## 🧭 **Spis treści**

### 📂 [1. Wczytanie danych oraz import wymaganych bibliotek](#📂-1-wczytanie-danych-oraz-import-wymaganych-bibliotek)  
📌 *Rozdział zawiera import niezbędnych bibliotek oraz wczytanie zbioru danych do ramki danych typu Pandas w celu dalszej analizy.*

### 📊 [2. Analiza i opis danych źródłowych](#📊-2-analiza-i-opis-danych-źródłowych)  
📌 *Analiza struktury danych, typów kolumn oraz wstępne statystyki opisowe. Identyfikacja problemów jakościowych.*

### 📉 [3. Wnioski z analizy danych dotyczące ich jakości w zakresie istotnym dla przygotowywanego raportu](#📉-3-wnioski-z-analizy-danych-dotyczące-ich-jakości-w-zakresie-istotnym-dla-przygotowywanego-raportu)  
📌 *Podsumowanie najważniejszych problemów wpływających na dalszą analizę.*

### 🛠️ [4. Założenia dotyczące możliwości skorygowania danych w celu poprawy ich jakości](#🛠️-4-założenia-dotyczące-możliwości-skorygowania-danych-w-celu-poprawy-ich-jakości)  
📌 *Określenie reguł czyszczenia danych, np. odrzucania błędnych wartości lokalizacji czy grupowania typów pojazdów.*

### 🧹 [5. Przetworzenie, wyczyszczenie i uzupełnienie danych](#🧹-5-przetworzenie-wyczyszczenie-i-uzupełnienie-danych)  
📌 *Zastosowanie reguł korekcyjnych, zamiana typów, odrzucenie niepoprawnych rekordów, uzupełnienie braków.*

### 💾 [6. Zapisanie skorygowanych danych](#💾-6-zapisanie-skorygowanych-danych)  
📌 *Zapis przetworzonych danych do pliku CSV do wykorzystania w dalszej analizie.*

### 🧾 [7. Podsumowanie części I projektu](#🧾-7-podsumowanie-części-i-projektu)  
📌 *Wnioski końcowe dotyczące jakości danych i ich przygotowania do analizy przestrzenno-czasowej.*


 ## 📂 **1. Wczytanie danych oraz import wymaganych bibliotek**
 ---

In [ ]:
import pandas as pd
import geopandas as gpd
#from geopy.geocoders import Nominatim
#from geopy.extra.rate_limiter import RateLimiter
import re
import branca as bc
import folium
from shapely.geometry import Point
import matplotlib.pyplot as plt

# Wymagane instalacje w terminalu:
# pip install geopandas pandas
# pip install branca
# pip install folium matplotlib mapclassify
# ściągnąć GeoPandas - nie używać labdy ani aplay nie używać,  spatial join to jest szybsze do przypisania punktu do dzielnicy
# engine = 'pyarrow'

In [ ]:
# Wczytanie danych źródłowych
Initial_data_df = pd.read_csv(r"Data/nypd-motor-vehicle-collisions/nypd-motor-vehicle-collisions.csv",
                             engine = "pyarrow")

In [ ]:
Initial_data_df.head(3)

## 🔍 **2. Analiza i opis danych źródłowych**
---

### 📦 **2.1 Rozmiar danych inicjalnych**


In [ ]:
Initial_data_df.shape 

### 🧾 **2.2 Ogólne informacje o danych występujących w poszczególnych kolumnach**


In [ ]:
Initial_data_df.info()

### 🔢 **2.3 Typy danych występujące w kolumnach**


In [ ]:
data_types_in_columns = {
    col: Initial_data_df[col].map(lambda x: type(x).__name__).unique().tolist()
    for col in Initial_data_df.columns
}

for col, types in data_types_in_columns.items():
    print(f"{col}: {types}")

### 📭 **2.4 Sprawdzenie czy istnieją puste wiersze bez podanych danych we wszystkich polach**


In [ ]:
# Sprawdzenie, które wiersze mają wszystkie pola puste
mask = Initial_data_df.isna().all(axis=1)

# Sprawdzenie, czy w ogóle takie wiersze istnieją:
any_empty_rows = mask.any()
print("Czy istnieją puste wiersze?:", any_empty_rows)

# Wyświetlenie tych wierszy (jeśli są)
#empty_rows = Initial_data_df[mask]

#print("Wiersze z wszystkimi wartościami NaN:")
#print(empty_rows)

### 🧬 **2.5 Sprawdzenie unikalności danych**


In [ ]:
# Powtórzone wiersze (takie same wartości we wszystkich polach wierszy)
Initial_data_df[Initial_data_df.duplicated()].head(3)

In [ ]:
# Przykładowy powielony wiersz
Initial_data_df[Initial_data_df["COLLISION_ID"] == 3725834]

In [ ]:
# Liczba powielonych wierszy
Repeated_lines_sr = Initial_data_df['COLLISION_ID'].value_counts(dropna = False)
print("Liczba powtarzających się wierszy:", len(Repeated_lines_sr[Repeated_lines_sr > 1]))

### ⏰ **2.6 Dane dotyczące czasu wystąpienia wypadku**


In [ ]:
Initial_data_df.loc[:2, ["ACCIDENT DATE", "ACCIDENT TIME"]]

### 🗺️ **2.7 Dane dotyczące lokalizacji miejsca wypadku**


#### 2.7.1 Przykładowe dane dotyczące lokalizacji wypadku

In [ ]:
Initial_data_df.iloc[:3, 2:10]

#### 2.7.2 Dzielnice Nowego Jorku, których dotyczą dane o wypadkach

In [ ]:
#Nazwy dzielnic, które występują w danych
pd.Series(Initial_data_df["BOROUGH"].unique())

#### 2.7.3 Braki w danych o miejscu wystąpienia wypadku 

In [ ]:
# Przykładowe wiersze, w których brakuje nazwy dzielnic
Initial_data_df[Initial_data_df["BOROUGH"].isnull()].head(3)

In [ ]:
#Liczba wierszy bez podanych nazw dzielnic
print('Liczba wierszy bez podanej nazwy dzielnicy:', len(Initial_data_df[Initial_data_df["BOROUGH"].isnull()]))

In [ ]:
# Liczba wierszy bez podanych nazw dzielnic, ale zawierających dane lokalizacyjne miejsca wypadku
# Wybranie wierszy bez nazwy dzielnicy i tylko z podanymi współrzędnymi
Initial_data_df[Initial_data_df["BOROUGH"].isna() &
Initial_data_df["LATITUDE"].notna() & 
Initial_data_df["LONGITUDE"].notna()][["BOROUGH", "LATITUDE", "LONGITUDE", "LOCATION"]].head(3)

In [ ]:
# Wiersze z brakującymi nazwami dzielnic, długością i szerokością geograficzną, ale z podaną wartością w kolumnie "LOCATION"
Initial_data_df[Initial_data_df["BOROUGH"].isna() &
Initial_data_df["LATITUDE"].isna() & 
Initial_data_df["LONGITUDE"].isna() &
Initial_data_df["LOCATION"].notna()][["BOROUGH", "LATITUDE", "LONGITUDE", "LOCATION"]].head(3)

#### 2.7.4 Współrzędne miejsc wypadków

In [ ]:
#Location_df = Corrected_data_df[["LATITUDE", "LONGITUDE"]]
Initial_data_df[['LATITUDE', 'LONGITUDE']].dropna().sample(5)

In [ ]:
# Zakres współrzędnych długości i szerokości geograficznej
print("Zakres Latitude: od", Initial_data_df['LATITUDE'].min(), "do", Initial_data_df['LATITUDE'].max())
print("Zakres Longitude: od", Initial_data_df['LONGITUDE'].min(), "do", Initial_data_df['LONGITUDE'].max())

##### 2.7.4.1 Układ współrzędnych danych lokalizacji wypadków

In [ ]:
# Wybranie wierszy bez nazwy dzielnicy i tylko z podanymi współrzędnymi
Missing_borough_df = Initial_data_df[Initial_data_df['BOROUGH'].isna() & Initial_data_df['LATITUDE'].notna() & Initial_data_df['LONGITUDE'].notna()]

# Utworzenie geometrii dla miejsc wypadków (listy punków w przestrzeni na podstawie współrzędnych długości i szerokości geograficznej)
Points_with_accident_coordinates_ls = [Point(xy) for xy in zip(Missing_borough_df['LONGITUDE'], Missing_borough_df['LATITUDE'])]

# Utworzenie zbioru danych o wypadkach z geometrią utowrzoną dla lokalizacji wypadków w postaci GeoDataFrame z przypisanym układem współrzędnych EPSG:4326
Accidents_gdf = gpd.GeoDataFrame(Missing_borough_df, geometry=Points_with_accident_coordinates_ls, crs="EPSG:4326")

# Sprawdzenie przypisanego układu
print("Układ współrzędnych, w którym zapisano długość i szerokość geograficzną miejsc wypadków:", Accidents_gdf.crs)

Kody EPSG to numeryczne identyfikatory przypisane do układów odniesienia przestrzennego (CRS – Coordinate Reference Systems), które są standaryzowane przez organizację European Petroleum Survey Group, dziś utrzymywaną przez OGP (International Association of Oil & Gas Producers).
Współrzędne źródłowe dotyczące wypadków w Nowym Jorku, jak można było się spodziewać podane są w układzie o identyfikatorze EPSG:4326 czyli WGS 84 – układzie używanym w GPS.

In [ ]:
# Przykładowe wiersze bez nazwy dzielnicy, zawierające współrzędne lokalizacji wypadków
Accidents_gdf.head(3)

In [ ]:
Accidents_gdf.info()

In [ ]:
# Wyświetlenie punktów lokalizacji wypadków
Accidents_gdf.explore()

#### 2.7.5 Kody pocztowe

W kolumnie ZIP CODE występują dane różnych typów liczbowych

### 🚑 **2.8 Dane dotyczące skutków i liczby poszkodowanych w wypadkach**


In [ ]:
# Dane o liczbie rannych i zabitych w wypadku
Initial_data_df.sort_values("NUMBER OF PERSONS KILLED", ascending=False).iloc[:10, 10:18]

### 💥 **2.9 Przyczyny wypadków**


In [ ]:
# Przykładowe dane dotyczące przyczyn wypadków
Initial_data_df.iloc[:50, 18:23]

### 🚗 **2.10 Typy pojazdów uczestniczących w wypadkach**


In [ ]:
Initial_data_df.iloc[:10, 23:]

In [ ]:
# Ramka danych z przyczynami wypadków
Vehicle_type_code_df  = Initial_data_df[["VEHICLE TYPE CODE 1",
                                          "VEHICLE TYPE CODE 2",
                                          "VEHICLE TYPE CODE 3",
                                          "VEHICLE TYPE CODE 4",
                                          "VEHICLE TYPE CODE 5"
                                         ]]

### 📉 **2.11 Zastawienie liczbowe i procentowe braków w danych źródłowych**


In [ ]:
#Liczba brakujących danych w poszczególnych kolumnach
Missing_date_df = Initial_data_df.isnull().sum()
Missing_date_df = Missing_date_df.to_frame(name = "Number of missing data")

In [ ]:
#Procent brakujących danych
Missing_percentage_df = Initial_data_df.isnull().sum()*100/len(Initial_data_df)
Missing_percentage_df = Missing_percentage_df.round(2).to_frame(name = "Percentage of missing data [%]")

In [ ]:
#Złączenie ramek
Missing_date_df.join(Missing_percentage_df)

## 📊 **3. Wnioski z analizy jakości danych w zakresie istotnym dla przygotowywanego raportu**
---

* Nie ma w danych źródłowych pustych wierszy
* Część wierszy powtarza się (mają te same wartości we wszystkich polach)
* Występuje duża liczba wierszy z danymi wypadku bez wskazanej nazwy dzielnicy
* Część wierszy, które nie mają podanej nazwy dzielnicy posiadają dane lokalizacyjne miejsca wypadku
* Wiersze, w których brakuje nazwy dzielnicy, długości i szerokości geograficznej nie posiadają również wypełnionej kolumny "LOCATION"
* Dane lokalizacyjne części wierszy są błędne i wskazują na miejsca poza granicami Nowego Jorku
* Liczby w kolumnach "ZIP CODE", "NUMBER OF PERSONS INJURED" oraz " NUMBER OF PERSONS KILLE" podane zostały w formacie dziesiętnym
* Część przyczyn wypadków o tym samym znaczeniu została zapisana pod różniącymi się od siebie nazwami

 ## 🛠️ **4. Założenia dotyczące możliwości skorygowania danych w celu poprawy ich jakości**
 ---

* Należy usunąć powtórzone wiersze aby nie wpływały na wyliczane statystyki
* Należy uzupełnić nazwy dzielnic w oparciu o położenie punktów wskazujących lokalizacje miejsc wypadków (w wierszach, dla których jest to możliwe)
* Jeżeli nazwa dzielnicy została podana w wierszu, to jest uznana za właściwą i nie jest kontrolowana w oparciu o połżenie punktu lokalizacji wypadku
* Nazwy dzielnic nie będą uzupełniane na podstawie nazw ulic - ulica może przebiegać przez więcej niż jedną dzielnicę, mogą wystąpić powtórzenia w nazwach ulic
* Wartości występujące w kolumnach "ZIP CODE", "NUMBER OF PERSONS INJURED" oraz " NUMBER OF PERSONS KILLE" należy zamienić na liczby całkowite
*  Należy ujednolicić nazwy przyczyn wypadków mające to samo znaczeni
* W celu polepszenia czytelności można zmienić format date

## 🧹 **5. Przetworzenie, wyczyszczenie i uzupełnienie danych**
---

In [ ]:
# Ramka danych na czyszczone, poprawiane i uzupełniane dane
Corrected_data_df = Initial_data_df

In [ ]:
Corrected_data_df.head(3)

In [ ]:
len(Corrected_data_df)

### 🧾 **5.1 Powtórzone wiersze pochodzące z nieprzetworzonych danych źródłowych**


In [ ]:
# Liczba powielonych wierszy
number_of_repeated_lines = Initial_data_df['COLLISION_ID'].value_counts(dropna = False)
print("Liczba powtarzających się identyfikatorów kolizji:", len(Repeated_lines_sr[Repeated_lines_sr > 1]))

In [ ]:
# Zliczenie powtórzonych wierszy
number_of_repeated_lines = Corrected_data_df.duplicated(keep=False).sum()
print("Liczba powtórzonych wierszy:", int(number_of_repeated_lines/2))

In [ ]:
# Usunięcie duplikatów wierszy na podstawie wartości z wszystkich kolumn - pól w całym wierszu
Corrected_data_df = Corrected_data_df.drop_duplicates()

In [ ]:
# Ponowne sprawdzenie czy występują powtórzone wiersze
number_of_repeated_lines = Corrected_data_df.duplicated(keep=False).sum()
print("Liczba powtórzonych wierszy:", int(number_of_repeated_lines/2))

In [ ]:
# Wyszukanie identyfikatorów kolizji, dla powielonych wierszy w danych źródłowych
Repeated_lines_sr.loc[Repeated_lines_sr > 1].head(3)

##### Sprawdzenie poprawności usunięcia duplikatów dla przykładowego wiersza (różnica między Initial_data_df a Initial_data_unique_df)

In [ ]:
# Dane źródłowe
Initial_data_df[Initial_data_df['COLLISION_ID'] == 268395]

In [ ]:
# Dane wyczyszczone z powtórzonych wierszy
Corrected_data_df[Corrected_data_df['COLLISION_ID'] == 268395]

#### Unikalność identyfikatorów kolizji

In [ ]:
# sprawdzenie unikalności danych w kolumnie COLLISION_ID
Corrected_data_df[Corrected_data_df["COLLISION_ID"].duplicated()]

### 📅 **5.2 Zmiana formatu dat**


In [ ]:
Corrected_data_df['ACCIDENT DATE'] = pd.to_datetime(Corrected_data_df['ACCIDENT DATE'])
Corrected_data_df['ACCIDENT DATE'] = Corrected_data_df['ACCIDENT DATE'].dt.strftime('%Y-%m-%d')

### 🔢 **5.3 Zmiana typu danych na liczby całkowite**


In [ ]:
Corrected_data_df["ZIP CODE"] = pd.to_numeric(Corrected_data_df["ZIP CODE"], errors='coerce').astype('Int64')

In [ ]:
Corrected_data_df["NUMBER OF PERSONS INJURED"] = pd.to_numeric(Corrected_data_df["NUMBER OF PERSONS INJURED"], errors='coerce').astype('Int64')

In [ ]:
Corrected_data_df["NUMBER OF PERSONS KILLED"] = pd.to_numeric(Corrected_data_df["NUMBER OF PERSONS KILLED"], errors='coerce').astype('Int64')

In [ ]:
# Kolumny ze zmienionymi typami danych
Corrected_data_df[["ZIP CODE", "NUMBER OF PERSONS INJURED", "NUMBER OF PERSONS KILLED"]].head()

In [ ]:
# Ponowne zliczenie powtórzonych wierszy po zmianie typów danych
number_of_repeated_lines = Corrected_data_df.duplicated(keep=False).sum()
print("Liczba powtórzonych wierszy:", int(number_of_repeated_lines/2))

In [ ]:
# Ponowne usunięcie duplikatów wierszy na podstawie wartości z wszystkich kolumn - pól w całym wierszu
Corrected_data_df = Corrected_data_df.drop_duplicates()

In [ ]:
# Sprawdzenie usunięcia powtórzonych wierszy
number_of_repeated_lines = Corrected_data_df.duplicated(keep=False).sum()
print("Liczba powtórzonych wierszy:", int(number_of_repeated_lines/2))

### 🧹 **5.4 Ujednolicenie nazw kategorii mających to samo znaczenie**


Połączenie pojęć oznaczających te same przyczyny

In [ ]:
Corrected_data_df.loc[Corrected_data_df["CONTRIBUTING FACTOR VEHICLE 1"].isin(['1', '80']), "CONTRIBUTING FACTOR VEHICLE 1"] = 'Unspecified'
Corrected_data_df.loc[Corrected_data_df["CONTRIBUTING FACTOR VEHICLE 2"].isin(['1', '80']), "CONTRIBUTING FACTOR VEHICLE 2"] = 'Unspecified'
Corrected_data_df.loc[Corrected_data_df["CONTRIBUTING FACTOR VEHICLE 3"].isin(['1', '80']), "CONTRIBUTING FACTOR VEHICLE 3"] = 'Unspecified'
Corrected_data_df.loc[Corrected_data_df["CONTRIBUTING FACTOR VEHICLE 4"].isin(['1', '80']), "CONTRIBUTING FACTOR VEHICLE 4"] = 'Unspecified'
Corrected_data_df.loc[Corrected_data_df["CONTRIBUTING FACTOR VEHICLE 5"].isin(['1', '80']), "CONTRIBUTING FACTOR VEHICLE 5"] = 'Unspecified'

In [ ]:
Corrected_data_df.loc[Corrected_data_df["CONTRIBUTING FACTOR VEHICLE 1"] == 'Drugs (Illegal)', "CONTRIBUTING FACTOR VEHICLE 1"] = 'Drugs (illegal)'
Corrected_data_df.loc[Corrected_data_df["CONTRIBUTING FACTOR VEHICLE 2"] == 'Drugs (Illegal)', "CONTRIBUTING FACTOR VEHICLE 2"] = 'Drugs (illegal)'
Corrected_data_df.loc[Corrected_data_df["CONTRIBUTING FACTOR VEHICLE 3"] == 'Drugs (Illegal)', "CONTRIBUTING FACTOR VEHICLE 3"] = 'Drugs (illegal)'
Corrected_data_df.loc[Corrected_data_df["CONTRIBUTING FACTOR VEHICLE 4"] == 'Drugs (Illegal)', "CONTRIBUTING FACTOR VEHICLE 4"] = 'Drugs (illegal)'
Corrected_data_df.loc[Corrected_data_df["CONTRIBUTING FACTOR VEHICLE 5"] == 'Drugs (Illegal)', "CONTRIBUTING FACTOR VEHICLE 5"] = 'Drugs (illegal)'

In [ ]:
Corrected_data_df.loc[Corrected_data_df["CONTRIBUTING FACTOR VEHICLE 1"] == 'Cell Phone (hand-Held)', "CONTRIBUTING FACTOR VEHICLE 1"] = 'Cell Phone (hand-held)'
Corrected_data_df.loc[Corrected_data_df["CONTRIBUTING FACTOR VEHICLE 2"] == 'Cell Phone (hand-Held)', "CONTRIBUTING FACTOR VEHICLE 2"] = 'Cell Phone (hand-held)'
Corrected_data_df.loc[Corrected_data_df["CONTRIBUTING FACTOR VEHICLE 3"] == 'Cell Phone (hand-Held)', "CONTRIBUTING FACTOR VEHICLE 3"] = 'Cell Phone (hand-held)'
Corrected_data_df.loc[Corrected_data_df["CONTRIBUTING FACTOR VEHICLE 4"] == 'Cell Phone (hand-Held)', "CONTRIBUTING FACTOR VEHICLE 4"] = 'Cell Phone (hand-held)'
Corrected_data_df.loc[Corrected_data_df["CONTRIBUTING FACTOR VEHICLE 5"] == 'Cell Phone (hand-Held)', "CONTRIBUTING FACTOR VEHICLE 5"] = 'Cell Phone (hand-held)'

In [ ]:
Corrected_data_df.loc[Corrected_data_df["CONTRIBUTING FACTOR VEHICLE 1"] == 'Illnes', "CONTRIBUTING FACTOR VEHICLE 1"] = 'Illness'
Corrected_data_df.loc[Corrected_data_df["CONTRIBUTING FACTOR VEHICLE 2"] == 'Illnes', "CONTRIBUTING FACTOR VEHICLE 2"] = 'Illness'
Corrected_data_df.loc[Corrected_data_df["CONTRIBUTING FACTOR VEHICLE 3"] == 'Illnes', "CONTRIBUTING FACTOR VEHICLE 3"] = 'Illness'
Corrected_data_df.loc[Corrected_data_df["CONTRIBUTING FACTOR VEHICLE 4"] == 'Illnes', "CONTRIBUTING FACTOR VEHICLE 4"] = 'Illness'
Corrected_data_df.loc[Corrected_data_df["CONTRIBUTING FACTOR VEHICLE 5"] == 'Illnes', "CONTRIBUTING FACTOR VEHICLE 5"] = 'Illness'

In [ ]:
# Sprawdzenie czy po ujednoliceniu nazw przyczyn wypadków nie pojawiły się powtórzone wiersze
number_of_repeated_lines = Corrected_data_df.duplicated(keep=False).sum()
print("Liczba powtórzonych wierszy:", int(number_of_repeated_lines/2))

In [ ]:
Corrected_data_df.info()

### 🗽 **5.5 Uzupełnienie brakujących nazw dzielnic Nowego Jorku**


In [ ]:
# Liczba wierszy bez wskazanej dzielnicy
number_of_rows_1 = Corrected_data_df["BOROUGH"].isnull().sum()
print("Liczba wierszy bez określonej dzielnicy:",number_of_rows_1)

In [ ]:
# Liczba wierszy bez wskazanej dzielnicy z podaną lokalizacją wypadku w postaci współrzędnych
number_of_rows_2 = Corrected_data_df[Corrected_data_df['BOROUGH'].isna() & Corrected_data_df['LATITUDE'].notna() & Corrected_data_df['LONGITUDE'].notna()]
print("Liczba wierszy z podaną lokalizacją bez określonej dzielnicy:",len(number_of_rows_2))

In [ ]:
# liczba wierszy, dla których nie będzie można uzupełnić nazwy dzielnicy
number_of_rows_3 = Corrected_data_df["BOROUGH"].isnull().sum() - len(Corrected_data_df[Corrected_data_df['BOROUGH'].isna() & Corrected_data_df['LATITUDE'].notna() & Corrected_data_df['LONGITUDE'].notna()])
print("Liczba wierszy, które pozostaną bez określonej dzielnicy:",number_of_rows_3)

In [ ]:
# Procent wierszy, dla których nie ma możliwości wyznaczenia nazwy dzielnicy na podstawie współrzędnych lokalizacji wypadku
print("W ujęciu procentowym:",((100 * number_of_rows_3)/len(Corrected_data_df)).round(1),"%")

In [ ]:
# Liczba brakujących danych w poszczególnych kolumnach
Missing_date_df = Corrected_data_df.isnull().sum()
Missing_date_df = Missing_date_df.to_frame(name = "Number of missing data")

# Procent brakujących danych
Missing_percentage_df = Corrected_data_df.isnull().sum()*100/len(Initial_data_df)
Missing_percentage_df = Missing_percentage_df.round(2).to_frame(name = "Percentage of missing data [%]")

# Złączenie ramek
Missing_date_df.join(Missing_percentage_df)

**W 23 % danych brakuje nazw dzielnic**

Korzystając z Pandas i współpracujących bibliotek Pythona, można na podstawie współrzędnych geograficznych (kolumn LATITUDE i LONGITUDE) uzupełnić brakujące informacje w kolumnie BOROUGH o dzielnicy Nowego Jorku.
Można to zrobić na kilka sposobów np. na podstawie wstecznego geokodowania (reverse geocoding), które pozwoli przypisać nazwę dzielnicy (BOROUGH) bazując na współrzędnych LATITUDE i LONGITUDE. Można też wykorzystać dopasowanie przestrzenne.

**W tym celu należy:**
1. W poprawianych danych usunąć wiersze bez współrzędnych i lokalizacji
2. Wczytać plik z geometrią granic dzielnic Nowego Jorku.
3. Stworzyć geometrię punktową na podstawie LATITUDE i LONGITUDE.
4. Sprawdzić zgodność układów współrzędnych dla obu zbiorów danych i jeżeli są różne, to wykonać transformację tak aby uzyskać zgodność.
5. Wykonać przestrzenne dopasowanie (sjoin) punktów do dzielnic.
6. Uzupełnić kolumnę BOROUGH na podstawie wyniku dopasowania.

#### 5.5.1 Przygotowanie danych wypadków

In [ ]:
from shapely.geometry import Point
#import geopandas as gpd

# Wybranie wierszy bez nazwy dzielnicy i tylko z podanymi współrzędnymi
Missing_borough_df = Corrected_data_df[Corrected_data_df['BOROUGH'].isna() & Corrected_data_df['LATITUDE'].notna() & Corrected_data_df['LONGITUDE'].notna()]

# Utworzenie geometrii dla miejsc wypadków (listy punków w przestrzeni płaskiej na podstawie współrzędnych długości i szerokości geograficznej)
Points_with_accident_coordinates_ls = [Point(xy) for xy in zip(Missing_borough_df['LONGITUDE'], Missing_borough_df['LATITUDE'])]

# Utworzenie zbioru danych o wypadkach z geometrią utowrzoną dla lokalizacji wypadków w postaci GeoDataFrame z przypisanym układem współrzędnych EPSG:4326
Accidents_gdf = gpd.GeoDataFrame(Missing_borough_df, geometry=Points_with_accident_coordinates_ls, crs="EPSG:4326")

# Sprawdzenie przypisanego układu
print(Accidents_gdf.crs)

#### 5.5.2 Wczytanie geometrii granic dzielnic Nowego Jorku

In [ ]:
# Załadowanie danych z granicami dzielnic NY z pliku SHP
Boroughs_gdf = gpd.read_file(r"Data/nybb_25a/nybb.shp")

In [ ]:
Boroughs_gdf.head(3)

**Nazwy dzielnic zapisano w inny sposób niż w danych źródłowych, gdzie pisane są wielkimi literami**

In [ ]:
Boroughs_gdf.info()

In [ ]:
# Wyświetlenie grafiki granic NY na podstawie pobranych danych
Boroughs_gdf.explore()

#### 5.5.3 Uzyskanie zgodności układów współrzędnych

##### Układ współrzędnych danych z geometrią granic Nowego Jorku

Sprawdzenie układu współrzędnych danych z geometrią granic Nowego Jork

In [ ]:
print(Boroughs_gdf.crs)

In [ ]:
Boroughs_gdf.crs

##### Zmiana układu współrzędnych geometrii dzielnic Nowego Jorku

Współrzędne lokalizacji miejsc wypadków w Nowym Jorku podane są w układzie **WGS 84 (EPSG:4326)**, a współrzędne z granicami dzielnic Nowego Yorku w  układzie **NAD83 / New York Long Island (ftUS) - EPSG:2263**. W związku z powyższym należy je przekonwertować tak aby były w jednakowym układzie.

In [ ]:
# Zmiana układu współrzędnych na WGS84 (EPSG:4326), aby pasował do LATITUDE i LONGITUDE
Boroughs_gdf = Boroughs_gdf.to_crs(epsg=4326)

Dla obliczania odległości dokładniejszy byłby metryczny układ EPSG:2263 (New York State Plane), ponieważ EPSG:4326 operuje na stopniach. Ale do celów tej analizy i o wizualizacj wystarczy EPSG:4326.

In [ ]:
print("Układ współrzędnych geometrii granic Nowego Jorku:", Boroughs_gdf.crs)

#### 5.5.4 Uzupełnienie brakujacych nazw dzielnic w danych o wypadkach w Nowym Jorku

Wiele punktów (lokalizacji miejsc wypadków), którym nie przypisano dzielnicy leży na granicy miasta.
To warunek przestrzenny **predicate='within'** w złączeniu przestrzennym **sjoin** odpowiada za to żeby zostały połączone tylko te rekordy, dla których geometria z Accidents_gdf (punkt) znajduje się w obrębie (wewnątrz) geometrii z Boroughs_gdf. Predicate "within" wymaga, by punkt znajdował się **całkowicie wewnątrz** geometrii (np. poligonu dzielnicy). Punkty na krawędzi nie spełniają tego warunku — dlatego wynik to NaN.
W celu przypisania ich do dzielnicy predicate zostanie zmieniony na **'intersects'** (punkt przecina poligon, czyli znajduje się wewnątrz lub na krawędzi) - dzięki temu zostaną przypisane również punkty leżące na granicy.


In [ ]:
# Nazwy dzielnic w danych z granicami
Boroughs_gdf["BoroName"].unique()

In [ ]:
# Zamiana nazw na pisownię wielkiemi literami
Boroughs_gdf["BoroName"] = Boroughs_gdf['BoroName'].str.upper()
Boroughs_gdf["BoroName"].unique()

In [ ]:
# Przypisanie dzielnicy do każdego punktu przy pomocy spatial join
# przestrzennego łączenia punktów z granicami dzielnicami w celu sprawdzenia, w której dzielnicy leży dany punkt
# i dołączenie kolumny BoroName z nazwą dzielnicy z granic dzielnic do punktów

Accident_points_with_borough_gdf = gpd.sjoin(Accidents_gdf, Boroughs_gdf[['BoroName', 'geometry']], how='left', predicate="within")

# Uzupełnienie brakujących wartości w kolumnie 'BOROUGH'
Corrected_data_df.loc[Accident_points_with_borough_gdf.index, 'BOROUGH'] = Accident_points_with_borough_gdf['BoroName']

Część punktów, którym nie przypisano dzielnicy leży na granicy miasta.
W celu przypisania ich do dzielnicy zostanie użyty warunek predicate="intersects" (punkt przecina poligon, czyli znajduje się wewnątrz lub na krawędzi).
Dzięki temu zostaną przypisane również punkty leżące na granicy. 
Aby analiza objęła również punkty leżące niedaleko granicy dodatkowo dane przestrzenne zotaną rozszerzone o niewielki **bufor**. 

In [ ]:
# Wybranie wierszy bez nazwy dzielnicy i tylko z podanymi współrzędnymi
Missing_borough_df = Corrected_data_df[Corrected_data_df['BOROUGH'].isna() & Corrected_data_df['LATITUDE'].notna() & Corrected_data_df['LONGITUDE'].notna()]

# Utworzenie geometrii dla miejsc wypadków (listy punków w przestrzeni płaskiej na podstawie współrzędnych długości i szerokości geograficznej)
Points_with_accident_coordinates_ls = [Point(xy) for xy in zip(Missing_borough_df['LONGITUDE'], Missing_borough_df['LATITUDE'])]

# Utworzenie zbioru danych o wypadkach z geometrią utowrzoną dla lokalizacji wypadków w postaci GeoDataFrame z przypisanym układem współrzędnych EPSG:4326
Accidents_gdf = gpd.GeoDataFrame(Missing_borough_df, geometry=Points_with_accident_coordinates_ls, crs="EPSG:4326")

# zmiana punktu w "kółko" o promieniu około ok. 100–120 m (0.001° ≈ 111 m) poprzez zbudowanie buffora
Accidents_gdf['geometry'] = Accidents_gdf.buffer(0.001)
Accident_points_with_borough_gdf = gpd.sjoin(Accidents_gdf, Boroughs_gdf[['BoroName', 'geometry']], how='left', predicate="intersects")

In [ ]:
Accident_points_with_borough_gdf.head(3)

In [ ]:
# Liczba punktów z podaną dzielicą
len(Accident_points_with_borough_gdf)

In [ ]:
# Nazwy dzielnic w danych z geometrią granic miasta
Boroughs_gdf["BoroName"].unique()

In [ ]:
# Uzupełnienie brakujących wartości w kolumnie 'BOROUGH'
Corrected_data_df.loc[Accident_points_with_borough_gdf.index, 'BOROUGH'] = Accident_points_with_borough_gdf['BoroName']

In [ ]:
Corrected_data_df.head(3)

In [ ]:
#Liczba brakujących danych w poszczególnych kolumnach
Missing_date_df = Corrected_data_df.isnull().sum()
Missing_date_df = Missing_date_df.to_frame(name = "Number of missing data")

#Procent brakujących danych
Missing_percentage_df = Corrected_data_df.isnull().sum()*100/len(Initial_data_df)
Missing_percentage_df = Missing_percentage_df.round(2).to_frame(name = "Percentage of missing data [%]")

#Złączenie ramek
Missing_date_df.join(Missing_percentage_df)

In [ ]:
# Sprawdzenie czy są i gdzie leżą punkty bez wskazanej dzielnicy mimo iż mają podane współrzędne miejsc wypadków

# Wybranie wierszy bez nazwy dzielnicy i tylko z podanymi współrzędnymi
Missing_borough_df = Corrected_data_df[Corrected_data_df['BOROUGH'].isna() & Corrected_data_df['LATITUDE'].notna() & Corrected_data_df['LONGITUDE'].notna()]

# Utworzenie geometrii dla miejsc wypadków (listy punków w przestrzeni płaskiej na podstawie współrzędnych długości i szerokości geograficznej)
Points_with_accident_coordinates_ls = [Point(xy) for xy in zip(Missing_borough_df['LONGITUDE'], Missing_borough_df['LATITUDE'])]

# Utworzenie zbioru danych o wypadkach bez wskazanej dzielnicy z geometrią dla lokalizacji wypadków w postaci GeoDataFrame z przypisanym układem współrzędnych EPSG:4326
Accidents_without_borough_gdf = gpd.GeoDataFrame(Missing_borough_df, geometry=Points_with_accident_coordinates_ls, crs="EPSG:4326")


In [ ]:
# Interaktywna mapa

# Warstwa z granicami dzielnic
m = Boroughs_gdf.explore(color="blue", style_kwds={'fillOpacity': 0, 'weight': 2}, tooltip="BoroName")

# Dodanie warstwy z wypadkami
Accidents_without_borough_gdf.explore(m=m, color="red", marker_kwds={'radius': 3}, tooltip="COLLISION_ID")

In [ ]:
Accidents_without_borough_gdf.head(3)

### 🚙 **5.6 Dane dotyczące typów pojazdów**


In [ ]:
# Kolumny z typami pojazdów, które uczestniczyły w wypadkach
Vehicle_columns = [
    'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2',
    'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5'
]

In [ ]:
# Unikalne typy pojazdów po konwersji na wielkie litery
Unique_vehicle_types_sr = pd.Series(dtype=str)
for col in Vehicle_columns:
    unique_values = Corrected_data_df[col].dropna().str.upper().unique()
    Unique_vehicle_types_sr = pd.concat([Unique_vehicle_types_sr, pd.Series(unique_values)])

# Usunięcie duplikatów i sortowanie
Unique_vehicle_types_sr = Unique_vehicle_types_sr.drop_duplicates().sort_values().reset_index(drop=True)

In [ ]:
# Liczba unikalnych typów pojazdów
len(Unique_vehicle_types_sr)

In [ ]:
# Wyeksportowanie typów pojazdów do pliku csv
Unique_vehicle_types_sr.to_csv(r"Data/unique_vehicle_types.csv", index=False)

**Wynik analizy:**
* dane zawierają wiele wartości nietypowych, prawdopodobnie błędnych (np. "0", "1", "2015", "11 PA")
* inne mogą być wariantami istniejących kategorii (np. "2 DR SEDAN", "4 DR SEDAN")

**Czynności do wykonania w celu poprawy jakości danych:**
* wartości ewidentnie błędne (np. cyfry, skróty niebędące nazwami pojazdów) zostaną umieszczone na liście wartości do odrzucenia - Excluded_values;
* warianty jednej nazwy (np. "2 DR SEDAN" → "SEDAN") zostaną pogrupowane w mapie normalizacji typów pajazdów - Reverse_vehicle_map;
* pozostałe – zostaną oznaczone jako "UNMAPPED" lub zachowane do dalszej klasyfikacji ręcznej.

In [ ]:
# Mapa normalizująca typy pojazdów
Reverse_vehicle_map = {
    # PASSENGER VEHICLE
    "SEDAN": "PASSENGER VEHICLE", "4DSD": "PASSENGER VEHICLE", "TAXI": "PASSENGER VEHICLE",
    "LIVERY": "PASSENGER VEHICLE", "COUPE": "PASSENGER VEHICLE", "CONVERTIBLE": "PASSENGER VEHICLE",
    "HATCHBACK": "PASSENGER VEHICLE", "2 DR SEDAN": "PASSENGER VEHICLE", "4 DR SEDAN"
    "TLC": "PASSENGER VEHICLE", "GOLF": "PASSENGER VEHICLE", "PASSENGER VEHICLE": "PASSENGER VEHICLE",

    # SUV
    "SUV": "SUV", "SPORT UTILITY / STATION WAGON": "SUV", "S.U.V.": "SUV", "S/U V": "SUV",

    # LIGHT TRUCK
    "PICK-UP TRUCK": "LIGHT TRUCK", "PICKUP TRUCK": "LIGHT TRUCK", "PICK UP TRUCK": "LIGHT TRUCK",
    "P/U": "LIGHT TRUCK", "PICK": "LIGHT TRUCK", "PICK-": "LIGHT TRUCK", "PICKU": "LIGHT TRUCK",
    "PK": "LIGHT TRUCK", "PKUP": "LIGHT TRUCK",

    # VAN
    "VAN": "VAN", "CARGO VAN": "VAN", "MINIVAN": "VAN", "MINI VAN": "VAN", "MINIV": "VAN", "LUNCH WAGON": "VAN",
    "C/V": "VAN", "VAN/TRUCK": "VAN", "MINI-VAN": "VAN", "M/V": "VAN","VAN (": "VAN", "VAN A": "VAN", "VAN C": "VAN",
    "VAN CAMPER": "VAN", "VAN F": "VAN", "VAN T": "VAN", "VAN W": "VAN", "VAN/T": "VAN",

    # TRUCK
    "BOX TRUCK": "TRUCK", "TRACTOR TRUCK": "TRUCK", "TRUCK": "TRUCK", "FLATBED": "TRUCK",
    "DUMP": "TRUCK", "MACK": "TRUCK", "MAC T": "TRUCK", "MULTI-WHEELED VEHICLE": "TRUCK",
    "TRACTOR": "TRUCK", "18 WHEELER" "DELIVERY TRUCK": "TRUCK", "BEVERAGE TRUCK": "TRUCK",
    "TRAC": "TRUCK", "TOW TRUCK": "TRUCK", "TOW TRUCK / WRECKER": "TRUCK", "TRAC": "TRUCK",
    "TRAC.": "TRUCK", "TRACK": "TRUCK", "TRACT": "TRUCK", "TRACTOR TRUCK DIESEL": "TRUCK",
    "TRACTOR TRUCK GASOLINE": "TRUCK",

    # BUS
    "BUS": "BUS", "SCHOOL BUS": "BUS", "COMMERCIAL BUS": "BUS", "INTERCITY BUS": "BUS",
    "TOUR BUS": "BUS", "CHARTER BUS": "BUS",

    # MOTORCYCLE
    "MOTORCYCLE": "MOTORCYCLE", "SCOOTER": "MOTORCYCLE", "MOTORSCOOTER": "MOTORCYCLE",
    "MOPED": "MOTORCYCLE", "MO-PE": "MOTORCYCLE", "MOPAD": "MOTORCYCLE", "MOPET": "MOTORCYCLE",
    "MOTORBIKE": "MOTORCYCLE", "E-SCOOTER": "MOTORCYCLE", "MC": "MOTORCYCLE", "M/C": "MOTORCYCLE",

    # BICYCLE
    "BICYCLE": "BICYCLE", "BIKE": "BICYCLE", "E-BIKE": "BICYCLE", "E-BICYCLE": "BICYCLE",
    "MINIBIKE": "BICYCLE", "MINICYCLE" "ELECTRIC BIKE": "BICYCLE", "BICYC"
    "EBIKE": "BICYCLE", "E- BI": "BICYCLE", "E-BIK": "BICYCLE", "E/BIK": "BICYCLE",

    # EMERGENCY
    "AMBULANCE": "EMERGENCY", "FIRE TRUCK": "EMERGENCY", "FIRET": "EMERGENCY",
    "FDNY": "EMERGENCY", "EMERGENCY VEHICLE": "EMERGENCY", "EMS": "EMERGENCY",
    "AMABU": "EMERGENCY", "AMBU": "EMERGENCY", "AMBUL": "EMERGENCY", "ANBUL": "EMERGENCY",

    # OTHER
    "MOTOR HOME": "OTHER", "MOTORIZED HOME": "OTHER", "MAIL": "OTHER", "MTA": "OTHER",
    "NYC D": "OTHER", "NYC M": "OTHER", "LTR": "OTHER", "CEME": "OTHER", "ME/BE": "OTHER",
    "MAN L": "OTHER", "U-HAUL": "OTHER", "UTILITY": "OTHER", "UTILITY TRAILER": "OTHER",
    "POSTAL": "OTHER",

    # Do odfiltrowania
    "UNKNOWN": None, "UNKOWN": None, "": None, "N/A": None, None: None
}


# Lista wartości do odrzucenia
Excluded_values = {
    "", "UNKNOWN", "UNKOWN", "OTHER", "N/A", None,
    "0", "00", "1", "11 PA", "2015", "985", "C/O"
}

# Czyszczenie danych i kategoryzacja
Cleaned_vehicles_sr = pd.concat([
    Corrected_data_df[col]
    .dropna()
    .str.upper()
    .map(Reverse_vehicle_map)
    for col in Vehicle_columns
])

# Filtrowanie nieistotnych
Cleaned_vehicles_sr = Cleaned_vehicles_sr[~Cleaned_vehicles_sr.isin(Excluded_values)]

# Zliczenie i procenty
Vehicle_counts_sr = Cleaned_vehicles_sr.value_counts()
Vehicle_percentages_sr = (Vehicle_counts_sr / Vehicle_counts_sr.sum() * 100).round(1)

# Wykres słupkowy z procentami wewnątrz
plt.figure(figsize=(10, 6))
bars = plt.barh(
    Vehicle_counts_sr.index[::-1],
    Vehicle_counts_sr.values[::-1],
    color='slateblue'
)

# Dodanie etykiet procentowych do środka słupków
for bar, percent in zip(bars, Vehicle_percentages_sr.values[::-1]):
    width = bar.get_width()
    plt.text(width / 2, bar.get_y() + bar.get_height() / 2,
             f"{percent:.1f}%", ha='center', va='center', color='white', fontsize=9)

plt.title("Typy pojazdów biorących udział w wypadkach")
plt.xlabel("Liczba wypadków")
plt.ylabel("Typ pojazdu")
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

**Pozostałe nierozpoznane wartości**

Zidentyfikowano jeszcze kilkaset nierozpoznanych nazw typów pojazdów (np. '(CEME', '2 HOR', '4DS', '3-WHE', '12 PA'). 
W wyniku dalszej analizy będzie można je:
* przypisać ręcznie do jednej z kategorii,
* potraktować jako dane niekwalifikujące się do analizy (i je odfiltrować),
* przeanalizować ich występowanie i znaczenie (jeśli są częste).

## 💾 **6. Zapisanie skorygowanych danych**
---

In [ ]:
Corrected_data_df.to_csv(r"Data/nypd-motor-vehicle-collisions/nypd-motor-vehicle-collisions-updated.csv", index=False)

In [ ]:
Corrected_data_df.info()

## ✅ **7. Podsumowanie części I projektu**
---

#### 📥 Sekcja: Wczytanie i podstawowa inspekcja danych

> ✅ Dane zostały pomyślnie zaimportowane i wstępnie zbadane pod kątem liczby rekordów, typów kolumn oraz podstawowych statystyk opisowych.  
> 📌 Zidentyfikowano brakujące wartości oraz kolumny potencjalnie nieprzydatne w dalszych analizach.


#### 🧹 Sekcja: Czyszczenie danych

> 🧽 Przeprowadzono szereg operacji czyszczących:  
> - usunięcie rekordów z błędnymi lub brakującymi datami,  
> - przekształcenie typów danych (np. daty/czasu),  
> - ujednolicenie formatów tekstowych i usunięcie pustych pól.

> 📉 Usunięto także dane spoza granic miasta, przygotowując zbiór do dalszych analiz przestrzennych.


#### 🌍 Sekcja: Lokalizacja i konwersja współrzędnych

> 📍 Dokonano konwersji danych lokalizacyjnych do formatu przestrzennego (GeoDataFrame).  
> 🗺️ Ustandaryzowano układ współrzędnych (EPSG:4326 i EPSG:2263) z myślą o analizie geograficznej i agregacji danych w dzielnicach.


#### 🧾 Sekcja: Finalne przygotowanie danych

> ✅ Dane zostały zapisane w ramce `Corrected_data_df` i zapisane do pliku CSV - gotowe do dalszej analizy.  
> 🔁 Uwzględniono zarówno rekordy z poprawną lokalizacją (dla analiz dzielnicowych), jak i dane bez współrzędnych (dla analiz ogólnych).


### 📊 **Końcowe wnioski z części I projektu**

> 🔎 Dane wejściowe wymagały istotnego przetworzenia, obejmującego:
> - identyfikację i usunięcie błędnych lokalizacji,  
> - oczyszczenie pól tekstowych i dat,  
> - konwersję danych do formatów przestrzennych.

> 🛠️ Dzięki tym operacjom przygotowano wiarygodny zbiór danych, gotowy do kompleksowej analizy w kolejnych etapach projektu (**Część II projektu**).  
> 📂 Dane zostały zorganizowane w sposób ułatwiający agregację i wizualizację w podziale na dzielnice, typy pojazdów, przyczyny wypadków oraz czas.
